In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [ ]:
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=config["FOUNDRY_CONNECTION_STRING"],
)

with open('../../docs/api.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

openapi = OpenApiTool(
    name="comfortai",
    spec=openapi_spec,
    description="HVAC Comfort Control Tool",
    auth=OpenApiAnonymousAuthDetails())

agent_prompt = '''
Use the comfortai tool to handle user requests regarding HVAC comfort control.

After invoking the tool, send the following in a response:

- a brief, friendly summary of what action you took to handle the request
- a bullet point list of data you received from successful tool invocation
- any error information you received from failed tool invocation
'''

project_client.agents.create_agent(
    model="gpt-4o",
    name="ComfortAI Assistant",
    instructions=agent_prompt,
    tools=openapi.definitions
)
